## IFT 3700 - 21 Décembre 2018 - Travail 2
### David Ogaus(1004183), Jonathan Caspar (20059041), Tommy Côté(1056362) et Xiuli Zhang (20039439)
------------
# Question 1
---------
## Préparation des données

In [2]:
import numpy as np
import re
import random
from sklearn.model_selection import train_test_split
%matplotlib inline

In [3]:
def pre_processing(file_name, test):
    file = open(file_name)
    rawData = list(file) # contient une liste de vecteurs
    regex = re.compile("\s*(0|1)\.") # expression régulière qui extrait chaque chiffre (0 ou 1) situé avant un point
    data, label = [], []

    for vector in rawData :
        parsedVector = re.findall(regex, vector)
        intVector = [int(pixel.strip()) for pixel in parsedVector]
        if (not test): label.append(intVector.pop(784)) # on retire le dernier bit correspondant à la classe du vecteur et on l'ajoute dans label
        data.append(intVector) # on ajoute le vecteur de pixels (auquel on a retiré le dernier bit) dans "data"
    if (test): return data
    return train_test_split(data, label, test_size=0)

In [4]:
X_train, X_test, y_train, y_test = pre_processing("PATCH.amat", False)
test_data = pre_processing("PATCH_test.amat", True)

## Réseau de neuronnes entièrement connectés

In [5]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-6, hidden_layer_sizes=(900), random_state=1)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-06, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=900, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [6]:
clf.score(X_test,y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Création de prediction.csv

In [10]:
prediction = np.asarray(clf.predict(test_data))
np.savetxt("prediction.csv", prediction, fmt="%d", delimiter=",")

# Question 2
---------
1. _**Trouvez les 1000 paires d’étoiles jumelles les plus proches (distance euclidienne de la position).**_
    
    Utilisons **MapReduce** :<br>
 
    Définition d'étoiles jumelles : leur **âge et leurs propriétés communes** laissent penser qu'elles seraient nées       ensemble, avant de disperser dans l'Univers.

    1. __Trouver les étoiles jumelles__ : 
        1. **"Mapping"** : pour chaque serveur, produire une liste de paires <clé, valeur> telles que:
            * la **clé** correspond à l'âge et les propriétés communes afin de vérifier si deux étoiles sont jumelles (servira au moment de l'étape "Regrouper")
            * la **valeur** correspond à son identité et sa position dans l'espace (afin de calculer les distances relatives plus tard)<br><br>

        2. **"Regrouper"** : les paires ayant la même clé sont des étoiles jumelles, on regroupe les paires par nouvelle clé commune
        
        3. **"Reduce"** : à partir des paires contenant plus d'une étoile dans leur champ "valeur", produire une nouvelle liste de paires <clé, valeur> telles que :
            * la **clé** correspond à un tuple des 2 étoiles jumelles (produit cartésien de toutes les étoiles si plus de 2 sont présentes)
            * la **valeur** correspond à leur distance relative<br><br>
            
    2. __Trouver les 1000 étoiles jumelles plus proches__
        4. **"Reduce"** : ordonner les paires obtenues dans l'étape C par distance en ordre croisant, choisir les 1000 premiers.
    
    
    
2. _**Comptez combien d'étoiles il y a dans chaque catégorie.**_

    On peut se servir à nouveau de **MapReduce** :<br><br>

    1. __Etape "Mapping"__ : on demande au 300 serveurs d'itérer sur chaque étoile qu'ils contiennent, à l'issue de ce parcours, ils doivent produire une liste de paires **<clé, valeur>** telles que :
        * la **clé** correspond à la valeur de la catégorie de chaque étoile (entre 1 et 10)
        * la **valeur** correspond au nombre de catégories trouvées pour chaque étoile (i.e. toujours 1 dans ce cas)<br><br>
    
    2. __Etape "Reduce"__ : chaque serveur doit réduire leurs listes résultantes en regroupant les paires ayant la même clé (chaque regroupement incrémente la valeur de la paire de 1) : mène à la production d'une liste de 1 à 10 paires (puisqu'une des 10 catégories pourrait ne pas apparaître sur un serveur en particulier) correspondant à chaque catégorie d'étoile et le nombre d'étoiles y appartenant pour **CHAQUE** serveur.<br><br>
    
    3. __Etape "Regroupement"__ : les 300 listes de 1 à 10 paires sont envoyées à un **serveur central** qui s'occupe de mettre bout à bout les listes (temps pratiquement linéaire)<br><br>
    
    4. __Etape "Reduce"__ : le serveur central doit réduire les listes comme dans l'étape 2.
    
    A la fin de ce traitement, la liste résultante est une liste contenant 10 paires correspondants à chaque catégorie d'étoiles et le nombre d'étoiles y appartenant en ayant pris en compte **TOUTES** les données contenues par les 300 serveurs.


3. _**Produisez un classifieur qui, étant donné le vecteur de caractéristiques (22 nombres réels), prédit la catégorie de l’étoile.**_<br><br>

    1. __Etape "Mapping"__ : on demande au 300 serveurs d'itérer sur chaque étoile qu'ils contiennent, à l'issue de ce parcours, ils doivent produire deux listes de vecteurs contenant respectivement les **22 nombres réels** et la **catégorie de chaque étoile".<br><br>
    
    2. Chaque serveur doit produire un classifieur KNN qu'il entraine avec la liste des vecteurs de 22 nombres réels et les "labels" de catégorie.<br><br>
    
    3. Quand toutes les classifieurs ont terminés leurs entrainements, on peut maintenant faire une **requête de prédiction** pour un vecteur donné : un serveur/noeud designé comme central (ayant accès au classifieur entrainé sur ses données mais aussi aux classifieurs des autres serveurs) reçoit un vecteur de 22 nombres réels :
        
        1) Le serveur central fait une prédiction du "label" du vecteur avec son propre classifieur
        
        2) Le serveur central envoie une requête aux 299 autres serveurs en leurs transmettant le vecteur et demande à ces autres serveurs de prédire le "label" du vecteur en se basant sur leurs classifieurs respectifs.
        
        3) Le serveur central récupère les 299 labels prédits et les ajoute au label qu'il avait prédit par lui-même.
        
        4) Le serveur central itère sur la liste des 300 "labels" prédits et retourne le **"label" majoritaire** (celui qui apparait le plus souvent)
    
    
    
    

# Question 3
---------

**Approche 1** : on entraine **$m$ (nombre de catégorie)** classifieurs binaires, chaque classifieur fait la classification entre une catégorie (classe positive) et toutes les autres catégories (classe négative). 

Par exemple, **$C_1$** fait la classification entre la **catégorie 1** et toutes les autres, **$C_2$** fait une classification entre la **catégorie 2** et toutes les autres etc...
On applique les **$m$ classifieurs** sur le jeux de données, on obtiendra **$m$ scores** : il suffit alors de choisir le score le plus élevé.


**Approche 2** : on entraine **$\frac{m(m-1)}{2}$** classifieurs binaires, chaque classifieur fait la classification entre une catégorie et une autre.

Par exemple, **$C_{1|2}$** fait la classification entre la **catégorie 1 et 2**, **$C_{1|3}$** fait la classification entre la **catégorie 1 et 3**, etc... 
On les applique sur le jeux de données, on obtient **$\frac{m(m-1)}{2}$ décisions**, on peut ensuite voter pour le meilleur classifieur.

Comparons les deux approches :

* L'approche 1 a des biais, parce que les données d'entraînement sont $1:m$, pour l'éviter on peut choisir $\frac{1}{3}$ de données de $(m-1)$ classes (les classes négatives).

* Dans l'approche 2 : s'il y a beaucoup de catégories, on doit entrainer beaucoup de modèles ce qui très coûteux.

Dans le cas à **3 ou 25 catégories** : on peut utiliser les deux approches, mais dans le cas à 25 catégories on choisit **1/3** de données des classes négatives.

Dans le cas à **12500 catégories** on doit utiliser l'approche 1 mais il serait plus judicieux et moins coûteux de choisir d'autres classifieurs comme des réseaux de neurones.



# Question 4
---------
On structure d'abord les données provenant de "**adult.csv**" :

In [13]:
import csv
import pandas as pd
i, j = 0, 0

data = open('adult.csv') 
reader = csv.reader(data)

dataSet=[] 
for row in reader:            
    dataSet.append(row)

print(len(dataSet))
def fonctionTb(k):
    
    datacompl2= []
    for k in range(k,k+1):
        for i in range(0,1):
    
            for j in range(0,15):
                age=dataSet[0][j]
                colonne=dataSet[k][j]
                donnee= age +':'+ colonne
                
                datacompl2.append(donnee)
                
    return datacompl2
dataClean= []
for l in range (len(dataSet)):
    resultat = fonctionTb(l)

    dataClean.append(resultat)

dataClean.pop(0)
data.close()

48843


On crée la fonction **createC1** qui prend une liste C1, la trie, mappe chaque élément de la liste sur **frozenset()** et renvoie cette liste de frozensets.

In [14]:
def createC1(dataClean):
    C1 = []
    for transaction in dataClean:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    #C1.sort()
    return list(map(frozenset, C1))#utilise un ensemble figé pour pouvoir l'utiliser comme clé dans un dictionnaire

In [15]:
D = list(map(set,dataClean))
D

[{'age:25',
  'capital-gain:0',
  'capital-loss:0',
  'education:11th',
  'educational-num:7',
  'fnlwgt:226802',
  'gender:Male',
  'hours-per-week:40',
  'income:<=50K',
  'marital-status:Never-married',
  'native-country:United-States',
  'occupation:Machine-op-inspct',
  'race:Black',
  'relationship:Own-child',
  'workclass:Private'},
 {'age:38',
  'capital-gain:0',
  'capital-loss:0',
  'education:HS-grad',
  'educational-num:9',
  'fnlwgt:89814',
  'gender:Male',
  'hours-per-week:50',
  'income:<=50K',
  'marital-status:Married-civ-spouse',
  'native-country:United-States',
  'occupation:Farming-fishing',
  'race:White',
  'relationship:Husband',
  'workclass:Private'},
 {'age:28',
  'capital-gain:0',
  'capital-loss:0',
  'education:Assoc-acdm',
  'educational-num:12',
  'fnlwgt:336951',
  'gender:Male',
  'hours-per-week:40',
  'income:>50K',
  'marital-status:Married-civ-spouse',
  'native-country:United-States',
  'occupation:Protective-serv',
  'race:White',
  'relationshi

La fonction **scanD** prend trois arguments: 
* **D** - un ensemble de données 
* **Ck** - une liste d'ensembles candidats 
* **minSupport** - un support min qui est le support minimum qui nous intéresse

Cette fonction est utilisée pour générer **L1** à partir de **C1**. 
De plus, elle retourne un **dictionnaire avec des valeurs de support**.

C1 est un groupe d’éléments candidat de taille 1. Dans l'algorithme **Apriori**, nous créons **C1**, puis nous analysons 
l'ensemble de données pour voir si ces ensembles d'éléments répondent à nos exigences de support minimales. 

Les ensembles d'éléments qui répondent à nos exigences minimales deviennent **L1**. 
**L1** est ensuite combiné pour devenir **C2** et **C2** sera filtré pour devenir **L2**.

In [16]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [18]:
C1 = createC1(dataClean)
L1,suppDat0 = scanD(D,C1,0.5)
L1

[frozenset({'race:White'}),
 frozenset({'income:<=50K'}),
 frozenset({'native-country:United-States'}),
 frozenset({'capital-loss:0'}),
 frozenset({'capital-gain:0'}),
 frozenset({'gender:Male'}),
 frozenset({'workclass:Private'})]

In [19]:
#La fonction aprioriGen () prendra une liste d'itemsets fréquents, Lk, et la taille des itemsets,
#k, pour produire Ck. Par exemple, les itemsets {0}, {1}, {2} et ainsi de suite seront générés {0,1} {0,2} et {1,2}.
def aprioriGen(Lk, k): #Creer Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #si k-2 premier elements sont egaux
                retList.append(Lk[i] | Lk[j]) #ajouter ensemble union
    return retList

def apriori(dataClean, minSupport = 0.5):
    C1 = createC1(dataClean)
    D = list(map(set, dataClean))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#balayer DB pour obtenir Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData #L contient des listes de jeux d'éléments fréquents qui ont rencontré un support minimum de 0.5. 
                          #La variable suppData est un dictionnaire avec les valeurs de support de nos itemsets
    
#La fonction generateRules () prend trois entrées: une liste d'ensembles d'éléments fréquents, 
#un dictionnaire de données de support pour ces ensembles d'éléments et un seuil de confiance minimal. 
#Il va générer une liste de règles avec des valeurs de confiance que nous pourrons trier plus tard.
def generateRules(L, supportData, minConf=0.7):  #supportData est un dictionnaire qui vient de la fonction scanD 
    bigRuleList = []
    for i in range(1, len(L)):#n'obtenez que les ensembles contenant deux objets ou plus
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList  

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    # H: items individuels dans le jeu d'associations 
    # brl:la grande liste de règles à modifier
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print (freqSet-conseq,'-->',conseq,'conf: %.2f' % conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [22]:
L,suppData = apriori(dataClean)
rules= generateRules(L,suppData, minConf=0.7)

frozenset({'workclass:Private'}) --> frozenset({'race:White'}) conf: 0.86
frozenset({'gender:Male'}) --> frozenset({'race:White'}) conf: 0.88
frozenset({'capital-gain:0'}) --> frozenset({'race:White'}) conf: 0.85
frozenset({'race:White'}) --> frozenset({'capital-gain:0'}) conf: 0.91
frozenset({'capital-loss:0'}) --> frozenset({'race:White'}) conf: 0.85
frozenset({'race:White'}) --> frozenset({'capital-loss:0'}) conf: 0.95
frozenset({'native-country:United-States'}) --> frozenset({'race:White'}) conf: 0.88
frozenset({'race:White'}) --> frozenset({'native-country:United-States'}) conf: 0.92
frozenset({'income:<=50K'}) --> frozenset({'race:White'}) conf: 0.84
frozenset({'race:White'}) --> frozenset({'income:<=50K'}) conf: 0.75
frozenset({'workclass:Private'}) --> frozenset({'capital-gain:0'}) conf: 0.92
frozenset({'gender:Male'}) --> frozenset({'capital-gain:0'}) conf: 0.91
frozenset({'workclass:Private'}) --> frozenset({'capital-loss:0'}) conf: 0.96
frozenset({'gender:Male'}) --> frozens

Voici les patrons trouvés avec les différentes règles d'associations trouvées, accompagnées de leurs niveaux de 
confiance respectifs.

**{} ----> {}** signifie que si on a trouvé des caractéristiques appartenant à une personne dans la première accolade : il est probable avec un certain niveau de confiance que les caractéristiques dans la seconde accolade appartiennent aussi à cette même personne.

* D'après l'étude des règles d'associations trouvées, on peut conclure que : la plupart des personnes qui ont un 'capital-gain:0', un 'capital-loss:0' et qui ont un revenu 'income:<=50K': sont de 'native-country:United-States' et de 'race:White', et proviennent de 'workclass:Private'


* Concernant la confidentialité des données, il n'y a pas grand chose à dire étant donné que les patrons trouvés ne fournissent pas assez d'informations pour permettre d'identifier un quelconque individu.


* Sur le plan éthique, on remarque qu'il y a des inégalités sociales entre les personnes surtout vis-à-vis de la race et de la classe de travail.

**En conclusion, le fait de découvrir et d'exploiter des patrons qui permettent de déduire certaines données à partir de la race, le genre, les origines des personnes, etc.. soulève des problèmes d'éthique puisque ce genre de déductions pourraient s'apparenter à du racisme, sexisme, et discrimination en règle général.**
       